In [2]:
#RunnablePassthrough(), 
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./rag_data/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a building where Winston Smith resides. It is a run-down apartment complex with a faulty lift, gritty dust, and a hallway that smells of boiled cabbage and old rag mats. The flat where Winston lives is on the seventh floor, and the building is poorly maintained due to the ongoing economy drive in preparation for Hate Week. The building has a telescreen that cannot be completely shut off, and a poster with the caption "BIG BROTHER IS WATCHING YOU" hangs on the wall.')

In [3]:
chain.invoke("주인공이 누구야?")

AIMessage(content='주인공은 윈스턴 스미스입니다.')

In [4]:
chain.invoke("왜?")

AIMessage(content='죄송합니다. 알려드릴 수 있는 정보가 없습니다.')

In [5]:
chain.invoke("왜 주인공이 윈스턴 스미스야?")

AIMessage(content='죄송합니다. 그 정보는 제가 알고 있는 내용에는 포함되어 있지 않습니다.')

In [8]:
#Map Reduce LCEL(LangChain Expression Languege) Chain | 
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./rag_data/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("How many ministries are mentioned")

AIMessage(content='Three ministries are mentioned in the text: the Ministry of Love, the Ministry of Plenty, and the Ministry of Truth.')

In [9]:
chain.invoke("where dose Winsthon go to work")


AIMessage(content='Winston goes to work at the Ministry of Truth.')

In [10]:
chain.invoke("Describe Winston")


AIMessage(content="Winston is a smallish, frail figure with fair hair, a naturally sanguine face, and roughened skin from coarse soap, blunt razor blades, and the cold winter weather. He wears blue overalls, the uniform of the party. Winston sets his features into an expression of quiet optimism when facing the telescreen, drinks Victory Gin, smokes Victory Cigarettes, and is drawn to a man named O'Brien. He feels uneasiness and fear mixed with hostility around a particular girl and experiences a constricted diaphragm when seeing the face of Goldstein.")

# RAG 시스템 코드 분석

## 1. 필요한 라이브러리 및 클래스 임포트

```python
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
```

이 부분에서는 LangChain 라이브러리에서 필요한 모든 클래스와 함수를 임포트합니다. 각 임포트의 역할은 다음과 같습니다:

- `ChatOpenAI`: OpenAI의 챗봇 모델을 사용하기 위한 클래스
- `UnstructuredFileLoader`: 구조화되지 않은 파일(텍스트 파일 등)을 로드하기 위한 클래스
- `CharacterTextSplitter`: 텍스트를 작은 청크로 분할하기 위한 클래스
- `OpenAIEmbeddings`, `CacheBackedEmbeddings`: 텍스트를 벡터로 변환하기 위한 클래스들
- `FAISS`: 벡터 데이터베이스를 생성하고 관리하기 위한 클래스
- `LocalFileStore`: 로컬 파일 시스템에 데이터를 저장하기 위한 클래스
- `ChatPromptTemplate`: 챗봇 프롬프트를 만들기 위한 클래스
- `RunnablePassthrough`, `RunnableLambda`: 데이터 처리 파이프라인을 구성하기 위한 클래스들

## 2. 언어 모델 초기화

```python
llm = ChatOpenAI(
    temperature=0.1,
)
```

OpenAI의 챗봇 모델을 초기화합니다. `temperature=0.1`로 설정하여 모델의 출력을 더 결정적(덜 무작위적)으로 만듭니다.

## 3. 캐시 디렉토리 설정

```python
cache_dir = LocalFileStore("./.cache/")
```

임베딩을 캐시하기 위한 로컬 디렉토리를 설정합니다. 이는 반복적인 계산을 줄이고 성능을 향상시키는 데 도움이 됩니다.

## 4. 텍스트 분할기 설정

```python
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
```

텍스트를 작은 청크로 나누기 위한 분할기를 설정합니다. 여기서는:
- 줄바꿈(`\n`)을 기준으로 텍스트를 나눕니다.
- 각 청크의 크기는 600 토큰입니다.
- 청크 간 100 토큰의 중복을 허용하여 문맥의 연속성을 유지합니다.

## 5. 문서 로드 및 분할

```python
loader = UnstructuredFileLoader("./rag_data/chapter_one.txt")
docs = loader.load_and_split(text_splitter=splitter)
```

`UnstructuredFileLoader`를 사용하여 텍스트 파일을 로드하고, 앞서 정의한 `splitter`를 사용하여 이를 작은 청크로 나눕니다.

## 6. 임베딩 설정

```python
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
```

OpenAI의 임베딩 모델을 초기화하고, 이를 캐시 기반 임베딩으로 래핑합니다. 이렇게 하면 이전에 계산된 임베딩을 재사용할 수 있어 효율성이 향상됩니다.

## 7. 벡터 데이터베이스 생성

```python
vectorstore = FAISS.from_documents(docs, cached_embeddings)
```

FAISS를 사용하여 문서의 벡터 표현을 저장하는 데이터베이스를 생성합니다. 이는 효율적인 유사성 검색을 가능하게 합니다.

## 8. 검색기(Retriever) 설정

```python
retriever = vectorstore.as_retriever()
```

벡터 데이터베이스를 검색기로 변환합니다. 이를 통해 질문과 관련된 문서를 효율적으로 검색할 수 있습니다.

## 9. 문서 매핑 프롬프트 설정

```python
map_doc_prompt = ChatPromptTemplate.from_messages([...])
```

각 문서 청크를 처리하기 위한 프롬프트 템플릿을 정의합니다. 이 프롬프트는 시스템과 사용자 메시지로 구성되며, 문서의 관련 부분을 추출하는 데 사용됩니다.

## 10. 문서 매핑 체인 설정

```python
map_doc_chain = map_doc_prompt | llm
```

프롬프트와 언어 모델을 연결하여 각 문서 청크를 처리하는 체인을 생성합니다.

## 11. 문서 매핑 함수 정의

```python
def map_docs(inputs):
    ...
```

이 함수는 검색된 모든 문서에 대해 `map_doc_chain`을 적용하고 결과를 결합합니다.

## 12. 매핑 체인 설정

```python
map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)
```

검색기와 `map_docs` 함수를 결합하여 전체 매핑 프로세스를 정의합니다.

## 13. 최종 프롬프트 설정

```python
final_prompt = ChatPromptTemplate.from_messages([...])
```

최종 답변을 생성하기 위한 프롬프트 템플릿을 정의합니다.

## 14. 전체 체인 구성

```python
chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm
```

모든 구성 요소를 하나의 체인으로 결합합니다. 이 체인은:
1. 질문을 받습니다.
2. 관련 문서를 검색합니다.
3. 검색된 문서에서 관련 정보를 추출합니다.
4. 최종 답변을 생성합니다.

## 15. 체인 실행

```python
chain.invoke("How many ministries are mentioned")
```

구성된 체인을 사용하여 질문에 대한 답변을 생성합니다.

이 코드는 문서에서 정보를 추출하고 질문에 답변하는 복잡한 RAG 시스템을 구현합니다. 각 구성 요소가 서로 연계되어 효율적이고 정확한 정보 검색 및 답변 생성 프로세스를 만듭니다.


# LangChain의 Runnable 컴포넌트 설명

## 1. langchain.schema.runnable

이는 LangChain에서 실행 가능한(runnable) 객체들의 기본 스키마를 정의하는 모듈입니다. 이 모듈은 다른 runnable 컴포넌트들의 기반이 되는 추상 클래스와 인터페이스를 제공합니다.

주요 특징:
- 실행 가능한 객체들의 기본 구조를 정의
- 다른 runnable 컴포넌트들이 상속받아 사용

## 2. RunnablePassthrough

RunnablePassthrough는 입력을 그대로 출력으로 전달하는 간단한 runnable 컴포넌트입니다.

주요 특징:
- 입력을 수정하지 않고 그대로 다음 단계로 전달
- 복잡한 체인에서 일부 데이터를 그대로 유지하고 싶을 때 유용
- 파이프라인의 중간에서 데이터를 보존하거나 우회시킬 때 사용

사용 예:
```python
from langchain.schema.runnable import RunnablePassthrough

passthrough = RunnablePassthrough()
result = passthrough.invoke("Hello, World!")
print(result)  # 출력: Hello, World!
```

## 3. RunnableLambda

RunnableLambda는 사용자 정의 함수를 runnable 객체로 변환하는 컴포넌트입니다.

주요 특징:
- 일반 파이썬 함수를 LangChain의 runnable 인터페이스에 맞게 래핑
- 복잡한 로직이나 데이터 변환을 체인에 쉽게 통합 가능
- 유연한 데이터 처리와 변환을 가능하게 함

사용 예:
```python
from langchain.schema.runnable import RunnableLambda

def uppercase_function(text):
    return text.upper()

uppercase_runnable = RunnableLambda(uppercase_function)
result = uppercase_runnable.invoke("hello, world!")
print(result)  # 출력: HELLO, WORLD!
```

## 실제 사용 시나리오

이러한 컴포넌트들은 주로 복잡한 LangChain 파이프라인을 구축할 때 사용됩니다. 예를 들어:

1. RunnablePassthrough를 사용하여 원본 질문을 파이프라인 전체에 걸쳐 유지
2. RunnableLambda를 사용하여 검색된 문서를 처리하거나 포맷팅
3. 이러한 컴포넌트들을 조합하여 질문-답변 시스템이나 문서 요약기 등을 구축

이 컴포넌트들을 활용하면 유연하고 강력한 자연어 처리 파이프라인을 구축할 수 있습니다.


# 파이썬의 Lambda 함수

Lambda 함수는 파이썬에서 작은 익명 함수를 만드는 방법입니다. '익명'이라는 말은 이 함수가 이름을 갖지 않는다는 의미입니다.

## 1. Lambda 함수의 특징

- 한 줄로 정의되는 간단한 함수입니다.
- 여러 개의 인자를 가질 수 있지만, 표현식은 하나만 가질 수 있습니다.
- 주로 다른 함수의 인자로 사용되거나 즉시 실행되는 로직에 사용됩니다.

## 2. Lambda 함수의 구조

기본 구조는 다음과 같습니다:

```python
lambda arguments: expression
```

- `lambda`: 키워드로, lambda 함수를 정의한다는 것을 나타냅니다.
- `arguments`: 함수의 인자들입니다. 여러 개일 수 있습니다.
- `expression`: 함수의 본문으로, 단일 표현식만 가능합니다.

## 3. Lambda 함수 예시

### 예시 1: 단순 계산
```python
# 제곱을 계산하는 lambda 함수
square = lambda x: x**2
print(square(5))  # 출력: 25
```

### 예시 2: 정렬에 사용
```python
# 리스트를 두 번째 요소를 기준으로 정렬
pairs = [(1, 'one'), (2, 'two'), (3, 'three'), (4, 'four')]
pairs.sort(key=lambda pair: pair[1])
print(pairs)  # 출력: [(4, 'four'), (1, 'one'), (3, 'three'), (2, 'two')]
```

### 예시 3: 함수 내에서 사용
```python
# map 함수와 함께 사용
numbers = [1, 2, 3, 4]
squared = list(map(lambda x: x**2, numbers))
print(squared)  # 출력: [1, 4, 9, 16]
```

## 4. Lambda vs 일반 함수

Lambda 함수는 간단한 연산에 유용하지만, 복잡한 로직에는 일반 함수가 더 적합합니다.

```python
# Lambda 함수
multiply = lambda x, y: x * y

# 동일한 기능의 일반 함수
def multiply(x, y):
    return x * y
```

Lambda 함수는 코드를 간결하게 만들 수 있지만, 과도한 사용은 가독성을 떨어뜨릴 수 있으므로 적절히 사용하는 것이 중요합니다.
